## Lakehouse Calendar Dimension
This notebook creates a calendar dimension (Also known as date dimension) for the lakehouse. It is intended to be reloaded daily, and defaults to loading data using a rolling 5 year period.

### Directions
- Set catalog and schema
- Update the table location
- Modify the date range as necessary by updating the dates CTE
- Add/modify/remove columns as necessary
- Schedule to run daily using Workflows

### References
- [Five Simple Steps for Implementing a Star Schema in Databricks With Delta Lake](https://www.databricks.com/blog/2022/05/20/five-simple-steps-for-implementing-a-star-schema-in-databricks-with-delta-lake.html)
- [Datetime Patterns for Formatting and Parsing](https://spark.apache.org/docs/latest/sql-ref-datetime-pattern.html)

In [0]:
use catalog identifier(:catalog);
use schema identifier(:schema);

In [0]:
create or replace table dim_calendar (
  date_int int not null comment 'Integer representation of the date',
  calendar_date date not null comment 'Calendar date',
  calendar_year int not null comment 'Calendar year',
  calendar_month string not null comment 'Calendar month (January, February, March)',
  month_of_year int not null comment 'Month of year (1 - 12)',
  calendar_day string not null comment 'Calendar day (Monday, Tuesday, Wednesday)',
  day_of_week int not null comment 'Day of week (1 - 7)',
  day_of_week_start_monday int not null comment 'Day of week starting Monday (1 - 7)',
  is_week_day string not null comment 'Flag for weekday, a day of the week other than Saturday or Sunday (Y, N)',
  day_of_month int not null comment 'Day of the month (1 - 31)',
  is_last_day_of_month string not null comment 'Flag for last day of the month (Y, N)',
  day_of_year int not null comment 'Day of the year (1 - 365)',
  week_of_year_iso int not null comment 'Week of the year (1 - 53)',
  quarter_of_year int not null comment 'Quarter of the year (1 - 4)',
  fiscal_year_oct_to_sep int not null comment 'Fiscal year for October to September',
  fiscal_month_oct_to_sep int not null comment 'Fiscal month for October to September (1 - 12)',
  fiscal_year_jul_to_jun int not null comment 'Fiscal year for July to June',
  fiscal_month_jul_to_jun int not null comment 'Fiscal month for July to June (1 - 12)',
  primary key (date_int) rely
)
comment 'Calendar dimension'
cluster by (date_int);

In [0]:
insert overwrite dim_calendar
--Set the date range in the dates CTE below
with dates as (
  select explode(sequence(current_date() - interval 5 years, current_date(), interval 1 day)) AS calendar_date
)
select
  year(calendar_date) * 10000 + month(calendar_date) * 100 + day(calendar_date) as date_int,
  calendar_date,
  year(calendar_date) AS calendar_year,
  date_format(calendar_date, 'MMM') as calendar_month,
  month(calendar_date) as month_of_year,
  date_format(calendar_date, 'EEEE') as calendar_day,
  dayofweek(calendar_date) AS day_of_week,
  weekday(calendar_date) + 1 as day_of_week_start_monday,
  case
    when weekday(calendar_date) < 5 then 'Y'
    else 'N'
  end as is_week_day,
  dayofmonth(calendar_date) as day_of_month,
  case
    when calendar_date = last_day(calendar_date) then 'Y'
    else 'N'
  end as is_last_day_of_month,
  dayofyear(calendar_date) as day_of_year,
  weekofyear(calendar_date) as week_of_year_iso,
  quarter(calendar_date) as quarter_of_year,
  /* Fiscal period examples */
  case
    when month(calendar_date) >= 10 then year(calendar_date) + 1
    else year(calendar_date)
  end as fiscal_year_oct_to_sep,
  (month(calendar_date) + 2) % 12 + 1 AS fiscal_month_oct_to_sep,
  case
    when month(calendar_date) >= 7 then year(calendar_date) + 1
    else year(calendar_date)
  end as fiscal_year_jul_to_jun,
  (month(calendar_date) + 5) % 12 + 1 AS fiscal_month_jul_to_jun
from
  dates
order by
  calendar_date;

num_affected_rows,num_inserted_rows
1827,1827


In [0]:
create or replace view vw_dim_calendar as
select * from dim_calendar;

In [0]:
optimize dim_calendar

path,metrics
s3://databricks-e2demofieldengwest/b169b504-4c54-49f2-bc3a-adf4b128f36d/tables/3582c165-48ee-4182-a1f8-e39a3a67b42d,"Map(recompressionCodec -> null, partitionsOptimized -> 0, numTableColumnsWithStats -> 18, skippedArchivedFiles -> 0, clusteringStats -> null, totalFilesSkipped -> 0, totalClusterParallelism -> 56, numFilesSkippedToReduceWriteAmplification -> 0, totalScheduledTasks -> 0, endTimeMs -> 1751058871897, startTimeMs -> 1751058867892, filesAdded -> Map(totalSize -> 0, avg -> 0.0, min -> null, totalFiles -> 0, max -> null), numBatches -> 0, numBins -> 0, deletionVectorStats -> Map(numDeletionVectorsRemoved -> 0, numDeletionVectorRowsRemoved -> 0), filesRemoved -> Map(totalSize -> 0, avg -> 0.0, min -> null, totalFiles -> 0, max -> null), totalTaskExecutionTimeMs -> 0, zOrderStats -> null, preserveInsertionOrder -> false, totalConsideredFiles -> 1, numBytesSkippedToReduceWriteAmplification -> 0, clusteringMetrics -> Map(numFilesSkippedAfterExpansion -> 0, totalLogicalSizeOfDataToRewriteInBytes -> 0, logicalSizeOfFilesClassifiedToIntermediateNodesInBytes -> 0, numLeafNodesCompacted -> 0, nodeMinNumFilesToCompact -> 4, isPOTriggered -> false, numSamplingJobs -> 0, clusterThresholdStrategy -> log, sizeOfTableInBytesBeforeLazyClustering -> 34956, numFilesClassifiedToLeafNodes -> 1, timeMetrics -> Map(metadataCreationTimeMs -> 1593, totalGetFilesForNodeCallsTimeMs -> 0, optimizerTimeMs -> 42, classifierTimeMs -> 79, totalSamplingTimeMs -> 0, metadataLoadTimeMs -> 0), approxClusteringQuality -> null, numThreadsForClassifier -> null, totalSizeOfFilesSkippedAfterExpansionInBytes -> 0, maxOptimizeBatchesInParallel -> 2, maxIterations -> 5, numLeafNodesExpanded -> 0, numIdealFilesWithTrimmedStringMaxValue -> 0, numCompactionTasksPlanned -> 0, numFilesClassifiedToIntermediateNodes -> 0, numLeafNodesClustered -> 0, minFileSize -> 16777216, numIdealFiles -> 0, currentIteration -> 1, numClusteringTasksPlanned -> 0, numGetFilesForNodeCalls -> 0, approxClusteringCoverage -> null, isNewMetadataCreated -> true, numIntermediateNodesCompacted -> 0, isFull -> false, totalLogicalSizeOfDataToCompactInBytes -> 0, totalSizeOfDataToRewriteInBytes -> 0, totalSizeOfDataToCompactInBytes -> 0, numIntermediateNodesClustered -> 0, maxFileSize -> 67108864, clusteringStrategy -> sizeAware, numOptimizeBatchesPlanned -> 0, totalLogicalSizeOfFilesSkippedAfterExpansionInBytes -> 0, numFilesSkippedWithoutStats -> 0, sizeOfFilesClassifiedToIntermediateNodesInBytes -> 0, sizeOfFilesClassifiedToLeafNodesInBytes -> 34956, approxClusteringQualityPerColumn -> null, numAddedFilesWithSameMinMaxOnClusteringColumns -> null, logicalSizeOfFilesClassifiedToLeafNodesInBytes -> 34956), numTableColumns -> 18, numFilesAdded -> 0, numFilesRemoved -> 0, autoCompactParallelismStats -> null)"


In [0]:
analyze table dim_calendar compute statistics for all columns;

In [0]:
vacuum dim_calendar

path
s3://databricks-e2demofieldengwest/b169b504-4c54-49f2-bc3a-adf4b128f36d/tables/3582c165-48ee-4182-a1f8-e39a3a67b42d
